<a href="https://colab.research.google.com/github/zhang99111/DD2360HT22/blob/main/hw_3/hw3_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [11]:
!nvcc -arch=sm_75 ./lab3_ex2.cu -o lab3_ex2

In [4]:
!./lab3_ex2 128 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
Host -> Device: 0.000099
Kernel: 0.000153
Device->Host: 0.000113
the commdan is correct

In [5]:
!./lab3_ex2 256 128 128 256

Input matrix dim (256 x 128) (128 x 256) (256 x 256)
Host -> Device: 0.000382
Kernel: 0.000268
Device->Host: 0.000394
the commdan is correct

In [6]:
!./lab3_ex2 512 128 128 512

Input matrix dim (512 x 128) (128 x 512) (512 x 512)
Host -> Device: 0.000284
Kernel: 0.000879
Device->Host: 0.001401
the commdan is correct

In [7]:
!./lab3_ex2 1024 128 128 1024

Input matrix dim (1024 x 128) (128 x 1024) (1024 x 1024)
Host -> Device: 0.000536
Kernel: 0.003171
Device->Host: 0.005285
the commdan is correct

In [8]:
!./lab3_ex2 2048 128 128 2048

Input matrix dim (2048 x 128) (128 x 2048) (2048 x 2048)
Host -> Device: 0.001001
Kernel: 0.012269
Device->Host: 0.021190
the commdan is correct

In [9]:
!./lab3_ex2 4096 128 128 4096

Input matrix dim (4096 x 128) (128 x 4096) (4096 x 4096)
Host -> Device: 0.001944
Kernel: 0.048726
Device->Host: 0.084496
the commdan is correct

In [12]:
!./lab3_ex2 128 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
float value
Host -> Device: 0.000076
Kernel: 0.000288
Device->Host: 0.000069
the commdan is correct

In [13]:
!./lab3_ex2 256 128 128 256

Input matrix dim (256 x 128) (128 x 256) (256 x 256)
float value
Host -> Device: 0.000095
Kernel: 0.000546
Device->Host: 0.000197
the commdan is correct

In [14]:
!./lab3_ex2 512 128 128 512

Input matrix dim (512 x 128) (128 x 512) (512 x 512)
float value
Host -> Device: 0.000155
Kernel: 0.001850
Device->Host: 0.000750
the commdan is correct

In [15]:
!./lab3_ex2 1024 128 128 1024

Input matrix dim (1024 x 128) (128 x 1024) (1024 x 1024)
float value
Host -> Device: 0.000295
Kernel: 0.006790
Device->Host: 0.003078
the commdan is correct

In [16]:
!./lab3_ex2 2048 128 128 2048

Input matrix dim (2048 x 128) (128 x 2048) (2048 x 2048)
float value
Host -> Device: 0.000570
Kernel: 0.026847
Device->Host: 0.010538
the commdan is correct

In [17]:
!./lab3_ex2 4096 128 128 4096

Input matrix dim (4096 x 128) (128 x 4096) (4096 x 4096)
float value
Host -> Device: 0.001007
Kernel: 0.106619
Device->Host: 0.042862
the commdan is correct

In [19]:
!/usr/local/cuda-11/bin/nv-nsight-cu-cli ./lab3_ex2 128 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
==PROF== Connected to process 462 (/content/drive/MyDrive/lab3_ex2)
float value
Host -> Device: 0.003174
==PROF== Profiling "gemm" - 1: 0%....50%....100% - 8 passes
Kernel: 0.827395
Device->Host: 0.000090
the commdan is correct==PROF== Disconnected from process 462
[462] lab3_ex2@127.0.0.1
  gemm(float*, float*, float*, int, int, int, int), 2022-Dec-15 22:44:41, Context 1, Stream 7
    Section: GPU Speed Of Light
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           4.99
    SM Frequency                                                             cycle/usecond                         584.27
    Elapsed Cycles                                                                   cycle                        154,194
    Memory [%]                            

In [20]:
!nvprof ./lab3_ex2 128 128 128 128

Input matrix dim (128 x 128) (128 x 128) (128 x 128)
==472== NVPROF is profiling process 472, command: ./lab3_ex2 128 128 128 128
float value
Host -> Device: 0.000087
Kernel: 0.000297
Device->Host: 0.000077
the commdan is correct==472== Profiling application: ./lab3_ex2 128 128 128 128
==472== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.48%  264.89us         1  264.89us  264.89us  264.89us  gemm(float*, float*, float*, int, int, int, int)
                    6.13%  17.758us         2  8.8790us  8.8630us  8.8950us  [CUDA memcpy HtoD]
                    2.39%  6.9120us         1  6.9120us  6.9120us  6.9120us  [CUDA memcpy DtoH]
      API calls:   99.64%  294.77ms         3  98.256ms  2.3500us  294.76ms  cudaMalloc
                    0.12%  341.06us         1  341.06us  341.06us  341.06us  cuDeviceTotalMem
                    0.09%  268.62us         2  134.31us  2.0740us  266.54us  cudaDeviceSynchronize
      

In [21]:
!./lab3_ex2 511 1023 1023 4094

Input matrix dim (511 x 1023) (1023 x 4094) (511 x 4094)
float value
Host -> Device: 0.004197
Kernel: 0.106374
Device->Host: 0.005237
error

In [22]:
!/usr/local/cuda-11/bin/nv-nsight-cu-cli ./lab3_ex2 511 1023 1023 4094

Input matrix dim (511 x 1023) (1023 x 4094) (511 x 4094)
==PROF== Connected to process 500 (/content/drive/MyDrive/lab3_ex2)
float value
Host -> Device: 0.004182
==PROF== Profiling "gemm" - 1: 0%....50%....100% - 8 passes
Kernel: 1.478402
Device->Host: 0.006223
error==PROF== Disconnected from process 500
[500] lab3_ex2@127.0.0.1
  gemm(float*, float*, float*, int, int, int, int), 2022-Dec-15 22:45:27, Context 1, Stream 7
    Section: GPU Speed Of Light
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/nsecond                           5.00
    SM Frequency                                                             cycle/usecond                         584.99
    Elapsed Cycles                                                                   cycle                     62,186,823
    Memory [%]                                         

In [ ]:
!nvprof ./lab3_ex2 2048 1536 1536 4096

Input matrix dim (2048 x 1536) (1536 x 4096) (2048 x 4096)
==284== NVPROF is profiling process 284, command: ./lab3_ex2 2048 1536 1536 4096
the commdan is correct==284== Profiling application: ./lab3_ex2 2048 1536 1536 4096
==284== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   79.03%  222.78ms         1  222.78ms  222.78ms  222.78ms  gemm(double*, double*, double*, int, int, int, int)
                   15.00%  42.291ms         1  42.291ms  42.291ms  42.291ms  [CUDA memcpy DtoH]
                    5.96%  16.804ms         2  8.4022ms  6.3323ms  10.472ms  [CUDA memcpy HtoD]
      API calls:   51.89%  309.06ms         3  103.02ms  136.62us  308.78ms  cudaMalloc
                   37.42%  222.84ms         1  222.84ms  222.84ms  222.84ms  cudaDeviceSynchronize
                   10.21%  60.822ms         3  20.274ms  6.5959ms  43.537ms  cudaMemcpy
                    0.36%  2.1447ms         3  714.90us  231.90us  1.142